In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import riskfolio as rp
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
from marketdatalib import map_scrip_to_yfin_ticker

In [ ]:
# Number of year of historic data to consider
years = 5

# The risk free return rate. This is often the return on a 10 year government bond.
# At present, the 10 year government bond rate is 6.84%. We are using 7.5% for the sake of simplicity which is the rate of gixed deposit.
risk_free_return = 0.075
acceptable_volatility = 0.15

# In intend to evaluate the performance of the portfolio, we will compare it with the performance of the benchmark index.
benchmark_index_nifty500 = 'NIFTY500_MULTICAP.NS'

scripts_to_ignore = ['SONACOMS']

special_scrips = ['ARE&M']

In [ ]:
holdings = pd.read_csv('../../data/MarketData/holdings.csv', names={'Instrument': 'Instrument', 'Quantity': 'Qty.', 'AvgCost': 'Avg. cost', 'LTP': 'LTP', 'CurVal': 'Cur. val', 'P&L': 'P&L', 'NetChg_Pct': 'Net chg.', 'DayChange_Pct': 'Day chg.', '':''}, header=0, index_col=0)
holdings = holdings.loc[:, holdings.columns != '']
holdings = holdings.drop(axis=0, index=scripts_to_ignore)

holdings['Weight'] = holdings['CurVal'] / holdings['CurVal'].sum()

portfolio_stocks = [map_scrip_to_yfin_ticker(scrip, 'NSE' if scrip not in special_scrips else 'BSE') for scrip in holdings.index]
# portfolio_stocks = [scrip for scrip in portfolio_stocks if scrip[0] != 'SONACOMS']
holdings

In [ ]:
master_data = pd.DataFrame()

start_date = datetime.today() - relativedelta(years=5)

for scrip, ticker in portfolio_stocks:
    scrip_data = pd.Series(yf.download(ticker.ticker, start=start_date, rounding=True)['Adj Close'], name=scrip)
    scrip_data.ffill(inplace=True)
    master_data = pd.concat([master_data, scrip_data], axis=1)

# master_data.ffill(inplace=True)
master_data.index = pd.to_datetime(master_data.index)
master_data = master_data.sort_index()
master_data.tail()

In [ ]:
def encich_with_returns(data: DataFrame):
    for scrip, ticker in portfolio_stocks:
        data[scrip + '_returns'] = data[scrip].pct_change(fill_method=None)
        data[scrip + '_log_returns'] = data[scrip].apply(lambda x: math.log(x)).diff()
    data.dropna(inplace=True, axis=0)
    return data

master_data = encich_with_returns(master_data)
master_data

In [ ]:
covariance_matrix = master_data[[scrip + '_returns' for scrip, ticker in portfolio_stocks]].cov().to_numpy()

In [ ]:
realized_returns = pd.DataFrame(index=[scrip for scrip, _ in portfolio_stocks], columns=['Returns', 'AnnualizedReturns', 'DailyVolatility', 'AnnualizedVolatility', 'SharpeRatio', 'SortinoRatio', 'Beta'])

for scrip, _ in portfolio_stocks:
    realized_returns.loc[scrip, 'Returns'] = (master_data[scrip + '_returns'] + 1).prod() - 1

    # Calculate average daily return
    avg_daily_return = np.mean(master_data[scrip + '_returns'])
    annualized_return = (1 + avg_daily_return).mean() ** 252 - 1
    # Calculate downside deviation
    downside_returns = master_data[scrip + '_returns'][master_data[scrip + '_returns'] < 0]
    downside_deviation = np.std(downside_returns)
    annualized_downside_deviation = downside_deviation * np.sqrt(252)

    daily_std = realized_returns.loc[scrip, 'DailyVolatility'] = (master_data[scrip + '_returns']).std()
    annualized_std = realized_returns.loc[scrip, 'AnnualizedVolatility'] = daily_std * math.sqrt(252)
    realized_returns.loc[scrip, 'AnnualizedReturns'] = annualized_return
    realized_returns.loc[scrip, 'SharpeRatio'] = (realized_returns.loc[scrip, 'AnnualizedReturns'] - risk_free_return) / annualized_std
    
    #TO: DO Calculate Beta. For this we need to calculate the covariance of the scrip with the benchmark index.
    # Need to figure out a way to get that data. I am thinking of using the Nifty 500 index as the benchmark index.
    realized_returns.loc[scrip, 'Beta'] = np.nan

    # Calculate Sortino Ratio
    sortino_ratio = (annualized_return - risk_free_return) / annualized_downside_deviation

    realized_returns.loc[scrip, 'SortinoRatio'] = sortino_ratio

realized_returns